## Régression linéaire et polynomiale pour le prix des citrouilles - Leçon 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infographie par Dasani Madipalli</figcaption>


#### Introduction

Jusqu'à présent, vous avez exploré ce qu'est la régression avec des données d'exemple issues du jeu de données sur les prix des citrouilles que nous utiliserons tout au long de cette leçon. Vous l'avez également visualisée en utilisant `ggplot2`. 💪

Vous êtes maintenant prêt à approfondir la régression pour l'apprentissage automatique. Dans cette leçon, vous allez en apprendre davantage sur deux types de régression : *la régression linéaire de base* et *la régression polynomiale*, ainsi que sur certaines notions mathématiques sous-jacentes à ces techniques.

> Tout au long de ce programme, nous supposons une connaissance minimale des mathématiques et cherchons à les rendre accessibles aux étudiants venant d'autres domaines. Soyez attentif aux notes, 🧮 aux encadrés, aux diagrammes et à d'autres outils pédagogiques pour faciliter la compréhension.

#### Préparation

Pour rappel, vous chargez ces données afin de poser des questions à leur sujet.

-   Quel est le meilleur moment pour acheter des citrouilles ?

-   Quel prix puis-je attendre pour une caisse de citrouilles miniatures ?

-   Devrais-je les acheter dans des paniers d'un demi-boisseau ou dans des boîtes de 1 1/9 boisseau ? Continuons à explorer ces données.

Dans la leçon précédente, vous avez créé un `tibble` (une réinvention moderne du tableau de données) et l'avez rempli avec une partie du jeu de données original, en standardisant les prix par boisseau. Cependant, en faisant cela, vous n'avez pu recueillir qu'environ 400 points de données et uniquement pour les mois d'automne. Peut-être pouvons-nous obtenir un peu plus de détails sur la nature des données en les nettoyant davantage ? Nous verrons... 🕵️‍♀️

Pour cette tâche, nous aurons besoin des packages suivants :

-   `tidyverse` : Le [tidyverse](https://www.tidyverse.org/) est une [collection de packages R](https://www.tidyverse.org/packages) conçue pour rendre la science des données plus rapide, plus facile et plus amusante !

-   `tidymodels` : Le [framework tidymodels](https://www.tidymodels.org/) est une [collection de packages](https://www.tidymodels.org/packages/) pour la modélisation et l'apprentissage automatique.

-   `janitor` : Le [package janitor](https://github.com/sfirke/janitor) fournit de petits outils simples pour examiner et nettoyer des données désordonnées.

-   `corrplot` : Le [package corrplot](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) offre un outil visuel exploratoire sur les matrices de corrélation qui prend en charge le réordonnancement automatique des variables pour aider à détecter des motifs cachés entre les variables.

Vous pouvez les installer avec la commande suivante :

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Le script ci-dessous vérifie si vous avez les packages nécessaires pour compléter ce module et les installe pour vous s'ils sont manquants.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Nous allons ensuite charger ces formidables packages et les rendre disponibles dans notre session R actuelle. (Ceci est simplement une illustration, `pacman::p_load()` l'a déjà fait pour vous)

## 1. Une ligne de régression linéaire

Comme vous l'avez appris dans la Leçon 1, l'objectif d'un exercice de régression linéaire est de pouvoir tracer une *ligne* *de* *meilleur ajustement* pour :

-   **Montrer les relations entre les variables**. Illustrer la relation entre les variables.

-   **Faire des prédictions**. Réaliser des prédictions précises sur la position d'un nouveau point de données par rapport à cette ligne.

Pour tracer ce type de ligne, nous utilisons une technique statistique appelée **Régression des Moindres Carrés**. Le terme `moindres carrés` signifie que tous les points de données entourant la ligne de régression sont élevés au carré, puis additionnés. Idéalement, cette somme finale est aussi petite que possible, car nous voulons un faible nombre d'erreurs, ou `moindres carrés`. Ainsi, la ligne de meilleur ajustement est celle qui nous donne la valeur la plus basse pour la somme des erreurs au carré - d'où le nom *régression des moindres carrés*.

Nous procédons ainsi car nous souhaitons modéliser une ligne ayant la plus faible distance cumulative par rapport à tous nos points de données. Nous élevons également les termes au carré avant de les additionner, car nous nous intéressons à leur magnitude plutôt qu'à leur direction.

> **🧮 Montrez-moi les calculs**
>
> Cette ligne, appelée *ligne de meilleur ajustement*, peut être exprimée par [une équation](https://fr.wikipedia.org/wiki/R%C3%A9gression_lin%C3%A9aire_simple) :
>
>     Y = a + bX
>
> `X` est la '`variable explicative` ou `prédicteur`'. `Y` est la '`variable dépendante` ou `résultat`'. La pente de la ligne est `b` et `a` est l'ordonnée à l'origine, qui correspond à la valeur de `Y` lorsque `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "pente = $y/x$")
    Infographie par Jen Looper
>
> Tout d'abord, calculez la pente `b`.
>
> En d'autres termes, et en se référant à la question initiale sur les données des citrouilles : "prédire le prix d'une citrouille par boisseau selon le mois", `X` ferait référence au prix et `Y` au mois de vente.
>
> ![](../../../../../../translated_images/calculation.989aa7822020d9d0ba9fc781f1ab5192f3421be86ebb88026528aef33c37b0d8.fr.png)
    Infographie par Jen Looper
> 
> Calculez la valeur de Y. Si vous payez environ 4 $, cela doit être en avril !
>
> Les calculs qui déterminent la ligne doivent démontrer la pente de la ligne, qui dépend également de l'ordonnée à l'origine, ou de la position de `Y` lorsque `X = 0`.
>
> Vous pouvez observer la méthode de calcul de ces valeurs sur le site [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Consultez également [ce calculateur des moindres carrés](https://www.mathsisfun.com/data/least-squares-calculator.html) pour voir comment les valeurs des nombres influencent la ligne.

Pas si effrayant, n'est-ce pas ? 🤓

#### Corrélation

Un autre terme à comprendre est le **Coefficient de Corrélation** entre les variables X et Y données. À l'aide d'un nuage de points, vous pouvez rapidement visualiser ce coefficient. Un graphique avec des points de données alignés de manière nette présente une forte corrélation, tandis qu'un graphique avec des points dispersés partout entre X et Y présente une faible corrélation.

Un bon modèle de régression linéaire sera celui qui a un Coefficient de Corrélation élevé (proche de 1 plutôt que de 0) en utilisant la méthode de Régression des Moindres Carrés avec une ligne de régression.


## **2. Une danse avec les données : créer un tableau de données pour la modélisation**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Illustration par @allison_horst</figcaption>


<!--![Illustration par \@allison_horst](../../../../../../translated_images/janitor.e4a77dd3d3e6a32e25327090b8a9c00dc7cf459c44fa9f184c5ecb0d48ce3794.fr.jpg){width="700"}-->


Chargez les bibliothèques nécessaires et le jeu de données. Convertissez les données en un tableau contenant un sous-ensemble des données :

-   Ne sélectionnez que les citrouilles dont le prix est indiqué par boisseau

-   Convertissez la date en mois

-   Calculez le prix comme une moyenne des prix élevés et bas

-   Ajustez le prix pour refléter la tarification par quantité de boisseaux

> Nous avons couvert ces étapes dans la [leçon précédente](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

Dans l'esprit de l'aventure pure, explorons le [`package janitor`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) qui offre des fonctions simples pour examiner et nettoyer des données désordonnées. Par exemple, examinons les noms de colonnes de nos données :


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Nous pouvons faire mieux. Transformons ces noms de colonnes en `friendR` en les convertissant au format [snake_case](https://en.wikipedia.org/wiki/Snake_case) à l'aide de `janitor::clean_names`. Pour en savoir plus sur cette fonction : `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Beaucoup de tidyR 🧹 ! Maintenant, une danse avec les données en utilisant `dplyr` comme dans la leçon précédente ! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Bon travail ! 👌 Vous avez maintenant un ensemble de données propre et bien organisé sur lequel vous pouvez construire votre nouveau modèle de régression !

Que diriez-vous d'un nuage de points ?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Un nuage de points nous rappelle que nous n'avons des données mensuelles que d'août à décembre. Nous aurons probablement besoin de plus de données pour pouvoir tirer des conclusions de manière linéaire.

Rejetons un coup d'œil à nos données de modélisation :


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Que faire si nous voulions prédire le `prix` d'une citrouille en fonction des colonnes `ville` ou `emballage`, qui sont de type caractère ? Ou encore plus simplement, comment pourrions-nous trouver la corrélation (qui nécessite que ses deux entrées soient numériques) entre, par exemple, `emballage` et `prix` ? 🤷🤷

Les modèles d'apprentissage automatique fonctionnent mieux avec des caractéristiques numériques plutôt que des valeurs textuelles, il est donc généralement nécessaire de convertir les caractéristiques catégorielles en représentations numériques.

Cela signifie que nous devons trouver un moyen de reformater nos prédicteurs pour les rendre plus faciles à utiliser efficacement par un modèle, un processus connu sous le nom de `ingénierie des caractéristiques`.


## 3. Prétraitement des données pour la modélisation avec recipes 👩‍🍳👨‍🍳

Les activités qui reformattent les valeurs des prédicteurs pour les rendre plus faciles à utiliser efficacement par un modèle sont appelées `ingénierie des caractéristiques`.

Différents modèles ont des exigences de prétraitement différentes. Par exemple, les moindres carrés nécessitent `l'encodage des variables catégorielles` telles que le mois, la variété et city_name. Cela consiste simplement à `traduire` une colonne avec des `valeurs catégorielles` en une ou plusieurs `colonnes numériques` qui remplacent la colonne d'origine.

Par exemple, supposons que vos données incluent la caractéristique catégorielle suivante :

|  ville   |
|:--------:|
| Denver   |
| Nairobi  |
|  Tokyo   |

Vous pouvez appliquer un *encodage ordinal* pour substituer une valeur entière unique à chaque catégorie, comme ceci :

| ville |
|:-----:|
|   0   |
|   1   |
|   2   |

Et c'est ce que nous allons faire avec nos données !

Dans cette section, nous allons explorer un autre package incroyable de Tidymodels : [recipes](https://tidymodels.github.io/recipes/) - conçu pour vous aider à prétraiter vos données **avant** d'entraîner votre modèle. Au cœur de ce package, une recette est un objet qui définit les étapes à appliquer à un ensemble de données afin de le préparer à la modélisation.

Maintenant, créons une recette qui prépare nos données à la modélisation en substituant un entier unique à toutes les observations dans les colonnes des prédicteurs :


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Super ! 👏 Nous venons de créer notre première recette qui spécifie un résultat (prix) et ses prédicteurs correspondants, et qui encode toutes les colonnes prédicteurs en un ensemble d'entiers 🙌 ! Décomposons cela rapidement :

-   L'appel à `recipe()` avec une formule indique à la recette les *rôles* des variables en utilisant les données `new_pumpkins` comme référence. Par exemple, la colonne `price` a été assignée au rôle de `outcome`, tandis que le reste des colonnes ont été assignées au rôle de `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` spécifie que tous les prédicteurs doivent être convertis en un ensemble d'entiers, avec une numérotation commençant à 0.

Nous sommes sûrs que vous vous posez des questions comme : "C'est tellement génial !! Mais que faire si je voulais vérifier que les recettes font exactement ce que je m'attends à ce qu'elles fassent ? 🤔"

C'est une excellente réflexion ! Vous voyez, une fois votre recette définie, vous pouvez estimer les paramètres nécessaires pour prétraiter les données, puis extraire les données traitées. Vous n'avez généralement pas besoin de faire cela lorsque vous utilisez Tidymodels (nous verrons la convention normale dans un instant -> `workflows`), mais cela peut être utile si vous voulez effectuer une sorte de vérification pour confirmer que les recettes fonctionnent comme prévu.

Pour cela, vous aurez besoin de deux autres verbes : `prep()` et `bake()`. Et comme toujours, nos petits amis R par [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) vous aident à mieux comprendre cela !

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Illustration par @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html) : estime les paramètres nécessaires à partir d'un ensemble d'entraînement qui peuvent ensuite être appliqués à d'autres ensembles de données. Par exemple, pour une colonne prédictive donnée, quelle observation sera assignée à l'entier 0, 1, 2, etc.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html) : prend une recette préparée et applique les opérations à n'importe quel ensemble de données.

Cela dit, préparons et appliquons nos recettes pour vraiment confirmer qu'en coulisses, les colonnes prédictives seront d'abord encodées avant qu'un modèle ne soit ajusté.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Youpi ! 🥳 Les données traitées `baked_pumpkins` ont tous leurs prédicteurs encodés, confirmant que les étapes de prétraitement définies comme notre recette fonctionnent comme prévu. Cela les rend plus difficiles à lire pour vous, mais beaucoup plus compréhensibles pour Tidymodels ! Prenez un moment pour découvrir quelle observation a été associée à un entier correspondant.

Il est également important de mentionner que `baked_pumpkins` est un tableau de données sur lequel nous pouvons effectuer des calculs.

Par exemple, essayons de trouver une bonne corrélation entre deux points de vos données afin de potentiellement construire un bon modèle prédictif. Nous utiliserons la fonction `cor()` pour cela. Tapez `?cor()` pour en savoir plus sur cette fonction.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Il s'avère qu'il n'y a qu'une faible corrélation entre la Ville et le Prix. Cependant, il existe une meilleure corrélation entre le Colis et son Prix. Cela a du sens, n'est-ce pas ? En général, plus la boîte de produits est grande, plus le prix est élevé.

Pendant qu'on y est, essayons également de visualiser une matrice de corrélation de toutes les colonnes en utilisant le package `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Bien mieux.

Une bonne question à poser maintenant à propos de ces données serait : '`Quel prix puis-je attendre d'un lot de citrouilles donné ?`' Allons-y !

> Note : Lorsque vous **`bake()`** la recette préparée **`pumpkins_prep`** avec **`new_data = NULL`**, vous extrayez les données d'entraînement traitées (c'est-à-dire encodées). Si vous aviez un autre ensemble de données, par exemple un ensemble de test, et que vous souhaitiez voir comment une recette le prétraiterait, vous pourriez simplement cuire **`pumpkins_prep`** avec **`new_data = test_set`**.

## 4. Construire un modèle de régression linéaire

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infographie par Dasani Madipalli</figcaption>


<!--![Infographie par Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.fr.png){width="800"}-->


Maintenant que nous avons créé une recette et confirmé que les données seront prétraitées correctement, construisons un modèle de régression pour répondre à la question : `Quel prix puis-je attendre d'un paquet de citrouilles donné ?`

#### Entraîner un modèle de régression linéaire en utilisant l'ensemble d'entraînement

Comme vous l'avez probablement déjà compris, la colonne *price* est la variable `outcome`, tandis que la colonne *package* est la variable `predictor`.

Pour ce faire, nous allons d'abord diviser les données de manière à ce que 80 % soient utilisées pour l'entraînement et 20 % pour le test, puis définir une recette qui encodera la colonne prédictive en un ensemble d'entiers, et enfin construire une spécification de modèle. Nous ne préparerons ni ne ferons cuire notre recette, car nous savons déjà qu'elle prétraitera les données comme prévu.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Bon travail ! Maintenant que nous avons une recette et une spécification de modèle, nous devons trouver un moyen de les regrouper dans un objet qui prétraitera d'abord les données (prep+bake en coulisses), ajustera le modèle sur les données prétraitées et permettra également des activités de post-traitement potentielles. De quoi vous rassurer, non ? 🤩

Dans Tidymodels, cet objet pratique s'appelle un [`workflow`](https://workflows.tidymodels.org/) et regroupe commodément vos composants de modélisation ! C'est ce que nous appellerions des *pipelines* en *Python*.

Alors, regroupons tout dans un workflow ! 📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

👌 De plus, un workflow peut être ajusté/entraîné de manière très similaire à un modèle.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

À partir des résultats du modèle, nous pouvons observer les coefficients appris lors de l'entraînement. Ils représentent les coefficients de la droite de meilleure ajustement qui minimise l'erreur globale entre les variables réelles et prédites.

#### Évaluer les performances du modèle avec le jeu de test

Il est temps de voir comment le modèle s'en est sorti 📏 ! Comment procéder ?

Maintenant que nous avons entraîné le modèle, nous pouvons l'utiliser pour faire des prédictions sur le `test_set` en utilisant `parsnip::predict()`. Ensuite, nous pouvons comparer ces prédictions aux valeurs réelles des étiquettes pour évaluer à quel point le modèle fonctionne bien (ou pas !).

Commençons par faire des prédictions pour le jeu de test, puis associons les colonnes au jeu de test.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Oui, vous venez de former un modèle et de l'utiliser pour faire des prédictions ! 🔮 Est-il performant ? Évaluons les résultats du modèle !

Dans Tidymodels, nous faisons cela en utilisant `yardstick::metrics()` ! Pour la régression linéaire, concentrons-nous sur les métriques suivantes :

-   `Root Mean Square Error (RMSE)` : La racine carrée de la [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Cela donne une métrique absolue dans la même unité que la variable cible (dans ce cas, le prix d'une citrouille). Plus la valeur est petite, meilleur est le modèle (de manière simpliste, cela représente le prix moyen par lequel les prédictions sont erronées !).

-   `Coefficient of Determination (souvent appelé R-squared ou R2)` : Une métrique relative où une valeur plus élevée indique un meilleur ajustement du modèle. Essentiellement, cette métrique représente la proportion de la variance entre les valeurs prédites et réelles que le modèle est capable d'expliquer.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Là va la performance du modèle. Voyons si nous pouvons obtenir une meilleure indication en visualisant un nuage de points du package et du prix, puis en utilisant les prédictions pour superposer une ligne de tendance.

Cela signifie que nous devrons préparer et transformer le jeu de test afin de coder la colonne package, puis l'associer aux prédictions faites par notre modèle.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Super ! Comme vous pouvez le constater, le modèle de régression linéaire ne généralise pas vraiment bien la relation entre un colis et son prix correspondant.

🎃 Félicitations, vous venez de créer un modèle qui peut aider à prédire le prix de quelques variétés de citrouilles. Votre champ de citrouilles pour les fêtes sera magnifique. Mais vous pouvez probablement créer un meilleur modèle !

## 5. Construire un modèle de régression polynomiale

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infographie par Dasani Madipalli</figcaption>


<!--![Infographie par Dasani Madipalli](../../../../../../translated_images/linear-polynomial.5523c7cb6576ccab0fecbd0e3505986eb2d191d9378e785f82befcf3a578a6e7.fr.png){width="800"}-->


Parfois, nos données peuvent ne pas avoir une relation linéaire, mais nous souhaitons tout de même prédire un résultat. La régression polynomiale peut nous aider à faire des prédictions pour des relations non linéaires plus complexes.

Prenons par exemple la relation entre le colis et le prix dans notre ensemble de données sur les citrouilles. Bien qu'il y ait parfois une relation linéaire entre les variables - plus le volume de la citrouille est grand, plus le prix est élevé - ces relations ne peuvent parfois pas être représentées par un plan ou une ligne droite.

> ✅ Voici [quelques exemples supplémentaires](https://online.stat.psu.edu/stat501/lesson/9/9.8) de données qui pourraient utiliser la régression polynomiale
>
> Regardez à nouveau la relation entre la variété et le prix dans le graphique précédent. Ce nuage de points semble-t-il devoir nécessairement être analysé par une ligne droite ? Peut-être pas. Dans ce cas, vous pouvez essayer la régression polynomiale.
>
> ✅ Les polynômes sont des expressions mathématiques qui peuvent comporter une ou plusieurs variables et coefficients

#### Entraîner un modèle de régression polynomiale en utilisant l'ensemble d'entraînement

La régression polynomiale crée une *courbe* pour mieux ajuster les données non linéaires.

Voyons si un modèle polynomial sera plus performant pour faire des prédictions. Nous suivrons une procédure quelque peu similaire à celle utilisée précédemment :

-   Créer une recette qui spécifie les étapes de prétraitement à effectuer sur nos données pour les préparer à la modélisation, c'est-à-dire : encoder les prédicteurs et calculer les polynômes de degré *n*

-   Construire une spécification de modèle

-   Regrouper la recette et la spécification de modèle dans un workflow

-   Créer un modèle en ajustant le workflow

-   Évaluer les performances du modèle sur les données de test

Allons-y !


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Évaluer les performances du modèle

👏👏 Vous avez construit un modèle polynomial, passons maintenant à la prédiction sur le jeu de test !


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo, évaluons les performances du modèle sur le test_set en utilisant `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Bien meilleure performance.

Le `rmse` est passé d'environ 7 à environ 3, ce qui indique une réduction de l'erreur entre le prix réel et le prix prédit. Vous pouvez *grossièrement* interpréter cela comme signifiant qu'en moyenne, les prédictions incorrectes se trompent d'environ 3 $. Le `rsq` est passé d'environ 0,4 à 0,8.

Tous ces indicateurs montrent que le modèle polynomial fonctionne bien mieux que le modèle linéaire. Beau travail !

Voyons si nous pouvons visualiser cela !


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Vous pouvez voir une courbe qui correspond mieux à vos données ! 🤩

Vous pouvez la rendre encore plus lisse en passant une formule polynomiale à `geom_smooth` comme ceci :


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Tout comme une courbe fluide !🤩

Voici comment faire une nouvelle prédiction :


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


La prédiction du `modèle polynomial` est logique, étant donné les diagrammes de dispersion de `price` et `package` ! Et, si ce modèle est meilleur que le précédent, en regardant les mêmes données, vous devez prévoir un budget pour ces citrouilles plus coûteuses !

🏆 Bravo ! Vous avez créé deux modèles de régression en une seule leçon. Dans la section finale sur la régression, vous apprendrez la régression logistique pour déterminer des catégories.

## **🚀Défi**

Testez plusieurs variables différentes dans ce notebook pour voir comment la corrélation correspond à la précision du modèle.

## [**Quiz après la leçon**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Révision & Étude personnelle**

Dans cette leçon, nous avons appris la régression linéaire. Il existe d'autres types importants de régression. Lisez à propos des techniques Stepwise, Ridge, Lasso et Elasticnet. Un bon cours pour approfondir est le [cours de Stanford sur l'apprentissage statistique](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Si vous souhaitez en savoir plus sur l'utilisation du formidable framework Tidymodels, veuillez consulter les ressources suivantes :

-   Site web de Tidymodels : [Commencez avec Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn et Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **MERCI À :**

[Allison Horst](https://twitter.com/allison_horst?lang=en) pour avoir créé les illustrations incroyables qui rendent R plus accueillant et engageant. Retrouvez plus d'illustrations dans sa [galerie](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de faire appel à une traduction humaine professionnelle. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
